### Récupération de la regression lineaire

In [58]:
import joblib
import numpy as np
from sklearn.linear_model import LinearRegression

model_path = './regression.joblib'
model = joblib.load(model_path)
model

LinearRegression()

### Recuperation des coefficients

In [13]:
coeffs, biais = list(model.coef_), model.intercept_
coeffs, biais

([1000.0000000000005, 10000.000000000005, 19999.999999999993],
 -5.820766091346741e-11)

### Generation du code C

In [54]:
def get_linear_prediction_c_code(model):
    C_code = "float coefs[] = {\n\t\t"
    for i in range(len(model.coef_)):
        C_code += str(model.coef_[i])
        if i < len(model.coef_) - 1:
              C_code += ', '
    C_code +=  ', ' + str(model.intercept_) + '\n\t};'
    code = """float prediction(float* features, int n_features)
{
""" + '\t' + C_code + '\n\tfloat res = 0;\n' + """\tfor (int i = 0; i < n_features; i++)
\t{
\t\tres += features[i] * coefs[i];
\t}
\tres += coefs[""" + str(len(model.coef_))+ """];
\treturn res;
}"""
    return code


In [51]:
print(get_linear_prediction_c_code(model))

float prediction(float* features, int n_features)
{
	float coefs[] = {
		1000.0000000000005, 10000.000000000005, 19999.999999999993, -5.820766091346741e-11
	};
	float res = 0;
	for (int i = 0; i < n_features; i++)
	{
		res += features[i] * coefs[i];
	}
	res += coefs[3];
	return to_return;
}


In [34]:
def get_linear_c_main(static_data):
    static_data_code = "\n\tfloat features[] = {\n\t\t"
    for i in range(len(static_data)):
        static_data_code += str(static_data[i])
        if i < len(static_data) - 1:
            static_data_code += ', '
    static_data_code += '\n\t};'

  
    code = """int main()
{""" + static_data_code + """
\tprintf("%f", prediction(features, """ + str(len(static_data)) + '));\n\treturn 0;' + """ 
}
  """
    return code

In [35]:
print(get_linear_c_main([1, 2, 3]))

int main()
{
	float features[] = {
		1, 2, 3
	};
	printf("%f", prediction(features, 3));
	return 0; 
}
  


### Génération du fichier C

In [59]:
def get_file(model, features):
    file = open("./code.c", "w+")
    file.write("#include <stddef.h>\n#include <stdio.h>\n" +get_linear_prediction_c_code(model) + '\n' + get_linear_c_main(features))
    file.close()
    print("gcc code.c -o prediction")


In [60]:
X = np.array([[8,1], [15, 20], [7, 6], [3, 4]])
y = np.dot(X, np.array([6, -1])) + 3.5
reg = LinearRegression().fit(X, y)

In [61]:
get_file(reg, [3, 5])

gcc code.c -o prediction


In [62]:
reg.predict([[3, 8]])

array([13.5])

En executant le code C généré, on obtient aussi 13.5.

In [63]:
get_file(reg, [3, 8])

gcc code.c -o prediction
